In [1]:
# %matplotlib widget

import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set_theme()

import torch 
import torch.nn as nn 
import torch.optim as optim

# import torch_optimizer as optim_custom
from torch.utils.data import Dataset, DataLoader
from utils.bernstein_torch import bernstein_coeff_order10_new
# import scipy.io as sio

# from models.mlp_qp_vis_aware_2 import MLP, vis_aware_track_net, PointNet
# import pol_matrix_comp
# from tqdm import trange

from models.learned_optim_qcqp_1 import PointNet, CustomGRULayer, GRU_Hidden_State, MLP_Pred, MLP_Init, Learned_QCQP
from scipy.io import loadmat
import pol_matrix_comp


In [2]:
# t_fin = 10.0
# num = 100
# tot_time = np.linspace(0, t_fin, num)
# tot_time_copy = tot_time.reshape(num, 1)


# P_np, Pdot_np, Pddot_np = pol_matrix_comp.pol_matrix_comp(tot_time_copy)

# nvar = np.shape(P_np)[1]

# P = torch.from_numpy(P_np).float()
# Pdot = torch.from_numpy(Pdot_np).float()
# Pddot = torch.from_numpy(Pddot_np).float()

# P_diag = torch.block_diag(P, P)
# Pddot_diag = torch.block_diag(Pddot, Pddot)



t_fin = 10.0
num = 100
tot_time = torch.linspace(0, t_fin, num)
tot_time_copy = tot_time.reshape(num, 1)
P, Pdot, Pddot = bernstein_coeff_order10_new(10, tot_time_copy[0], tot_time_copy[-1], tot_time_copy)
P_diag = torch.block_diag(P, P)
Pddot_diag = torch.block_diag(Pddot, Pddot)

nvar = np.shape(P)[1]
num_obs = 8

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")
torch.manual_seed(42)
torch.cuda.manual_seed(42)

Using cuda device


In [4]:
data = loadmat('./dataset/goal_reaching_dataset_4.mat')

init_state_ego = data['init_state_ego']
goal_des = data['goal_des']
closest_obs = data['closest_obs']
v_obs = data['v_obs']
y_lane_bound = data['y_lane_bound']
y_lb = y_lane_bound[:, 0]
y_ub = y_lane_bound[:, 1]
param_des = data['param_des']
dim_x_obs = data['dim_x_obs']
dim_y_obs = data['dim_y_obs']
psi_obs = data['psi_obs']

# print(np.shape(dim_x_obs))


dataset_size = np.shape(init_state_ego)[0]

inp = np.hstack(( init_state_ego, param_des, y_lane_bound   ))

inp_mean, inp_std = inp.mean(), inp.std()


pcd_data = closest_obs.reshape(dataset_size, 2, num_obs   )

min_pcd, max_pcd = pcd_data.min(), pcd_data.max()

pcd_mean, pcd_std = pcd_data.mean(), pcd_data.std()

In [5]:
# Custom Dataset Loader 
class TrajDataset(Dataset):
	"""Expert Trajectory Dataset."""
	def __init__(self, inp, init_state_ego, pcd_data, param_des, closest_obs, y_lane_bound, goal_des, psi_obs, dim_x_obs, dim_y_obs):
		
		# State Data
		self.inp = inp
  
		self.init_state_ego = init_state_ego
		
		# PCD Data
		self.pcd_data = pcd_data
		
		# Expert Coeff
		self.param_des = param_des
		
		self.closest_obs = closest_obs
  
		self.y_lane_bound = y_lane_bound
		self.goal_des = goal_des
		self.psi_obs = psi_obs 
		self.dim_x_obs = dim_x_obs 
		self.dim_y_obs = dim_y_obs

	
	def __len__(self):
		return len(self.inp)    
			
	def __getitem__(self, idx):
		
		# Inputs
		inp = self.inp[idx]
		init_state_ego = self.inp[idx]

		pcd_data = self.pcd_data[idx]
		
		# Outputs
		param_des = self.param_des[idx]
		
		closest_obs = self.closest_obs[idx]
  
		y_lane_bound = self.y_lane_bound[idx]
		goal_des = self.goal_des[idx]
		dim_x_obs = self.dim_x_obs[idx]
		dim_y_obs = self.dim_y_obs[idx] 
		psi_obs = self.psi_obs[idx]

  
  
		return torch.tensor(inp).float(), torch.tensor(init_state_ego).float(), torch.tensor(pcd_data).float(), torch.tensor(param_des).float(), torch.tensor(closest_obs).float(), torch.tensor(y_lane_bound).float(), torch.tensor(goal_des).float(),\
			   torch.tensor(psi_obs).float(), torch.tensor(dim_x_obs).float(), torch.tensor(dim_y_obs).float()		

# Batch Size - 3k or 4k
batch_size = 256

# pcd_data = pcd_data.reshape(data_set_size, 2, 200)

# Using PyTorch Dataloader
train_dataset = TrajDataset(inp, init_state_ego, pcd_data, param_des, closest_obs, y_lane_bound, goal_des, psi_obs, dim_x_obs, dim_y_obs)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True)


In [6]:
# Differentiable Layer
num_batch = train_loader.batch_size

P = P.to(device) 
Pdot = Pdot.to(device)
P_diag = P_diag.to(device)
Pddot_diag = Pddot_diag.to(device)

# PointNet
pcd_features = 40
point_net = PointNet(inp_channel=2, emb_dims=1024, output_channels=pcd_features)


mlp_pred_inp_dim = np.shape(inp)[1]+pcd_features
mlp_pred_hidden_dim = 256
mlp_pred_out_dim = 2*nvar 


mlp_init_inp_dim = np.shape(inp)[1]+pcd_features+2*nvar
mlp_init_hidden_dim = 256
mlp_init_out_dim = 4*nvar 


#########################33

gru_input_size = 12*nvar 
# print(gru_input_size)
gru_hidden_size = 512
gru_output_size = 4*nvar 
# gru_context_size = mlp_planner_inp_dim

gru_context = CustomGRULayer(gru_input_size, gru_hidden_size, gru_output_size)

input_hidden_state_init = pcd_features+np.shape(inp)[1]+2*nvar
mid_hidden_state_init = 512
out_hidden_state_init = gru_hidden_size

gru_hidden_state_init  =  GRU_Hidden_State(input_hidden_state_init, mid_hidden_state_init, out_hidden_state_init)


mlp_pred = MLP_Pred(mlp_pred_inp_dim, mlp_pred_hidden_dim, mlp_pred_out_dim  )
mlp_init = MLP_Init(mlp_init_inp_dim, mlp_init_hidden_dim, mlp_init_out_dim  )


model = Learned_QCQP(num_obs, t_fin, P, Pdot, Pddot, point_net, num_batch, min_pcd, max_pcd, inp_mean, inp_std, gru_context, gru_hidden_state_init, mlp_pred, mlp_init).to(device)



In [ ]:
epochs = 800
step = 0 
beta = 1.0 # 3.5

optimizer = optim.AdamW(model.parameters(), lr = 1e-3, weight_decay=6e-5)
# optimizer = optim.RMSprop(model.parameters(), lr = 2e-3, weight_decay=6e-5)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 100, gamma = 0.1)

avg_train_loss, avg_loss_primal, avg_fixed_point_loss, avg_res_loss, avg_steer_loss = [], [], [], [], []
model.train()
for epoch in range(epochs):
	
	# Train Loop
	losses_train, aug_losses_primal, aug_losses_fixed_point, aug_losses_res, aug_losses_steer = [], [], [], [], []
	
	for (inp, init_state_ego, pcd_data, param_des, closest_obs, y_lane_bound, goal_des, psi_obs, dim_x_obs, dim_y_obs) in train_loader:
		
		# Input and Output 
  
		################################################################################################

		# print(goal_des[0])
		# print(y_lb[0], y_ub[0])

  
		inp = inp.to(device)
		init_state_ego = init_state_ego.to(device)
		param_des = param_des.to(device)

		pcd_data = pcd_data.to(device)
		closest_obs = closest_obs.to(device)
		y_lane_bound = y_lane_bound.to(device)
		y_lb = y_lane_bound[:, 0]
		y_ub = y_lane_bound[:, 1]
		goal_des = goal_des.to(device)
		psi_obs = psi_obs.to(device)
		dim_x_obs = dim_x_obs.to(device)
		dim_y_obs = dim_y_obs.to(device)
  
		x_obs = closest_obs[:, 0:num_obs]
		y_obs = closest_obs[:, num_obs:2*num_obs]
		
  
		primal_sol, accumulated_res_primal, accumulated_res_fixed_point, c_pred = model(inp, init_state_ego, pcd_data,  closest_obs, psi_obs, dim_x_obs, dim_y_obs,  y_ub, y_lb)
		loss, fixed_point_loss, primal_loss, res_loss, steer_loss = model.ss_loss(accumulated_res_primal, accumulated_res_fixed_point, primal_sol, param_des, Pddot_diag, c_pred)
	 
		optimizer.zero_grad()
		loss.backward()
		optimizer.step()
		
		losses_train.append(loss.detach().cpu().numpy()) 
		aug_losses_primal.append(primal_loss.detach().cpu().numpy())
		aug_losses_fixed_point.append(fixed_point_loss.detach().cpu().numpy())
		aug_losses_res.append(res_loss.detach().cpu().numpy())
		aug_losses_steer.append(steer_loss.detach().cpu().numpy())

		
	# scale = scale*1.2	
		
	if epoch % 4 == 0:    
		print(f"Epoch: {epoch + 1}, Train Loss: {np.average(losses_train):.3f}, aug_loss_primal: {np.average(aug_losses_primal):.3f}, aug_loss_fixed_point: {np.average(aug_losses_fixed_point):.3f}, res_loss: {np.average(aug_losses_res):.3f}, steer_loss: {np.average(aug_losses_steer):.3f} ")

	step += 0.15 #0.15
	scheduler.step()
	avg_train_loss.append(np.average(losses_train)), avg_loss_primal.append(np.average(aug_losses_primal)), avg_fixed_point_loss.append(np.average(aug_losses_fixed_point)),  avg_res_loss.append(np.average(aug_losses_res)), avg_steer_loss.append(np.average(aug_losses_steer))
	
		



Epoch: 1, Train Loss: 12.847, aug_loss_primal: 3.185, aug_loss_fixed_point: 6.697, res_loss: 51.428, steer_loss: 1.007 
Epoch: 5, Train Loss: 4.027, aug_loss_primal: 0.286, aug_loss_fixed_point: 2.553, res_loss: 14.041, steer_loss: 0.070 
Epoch: 9, Train Loss: 3.230, aug_loss_primal: 0.190, aug_loss_fixed_point: 2.336, res_loss: 8.475, steer_loss: 0.046 
Epoch: 13, Train Loss: 3.562, aug_loss_primal: 0.284, aug_loss_fixed_point: 2.676, res_loss: 8.127, steer_loss: 0.073 
Epoch: 17, Train Loss: 2.983, aug_loss_primal: 0.156, aug_loss_fixed_point: 2.260, res_loss: 6.754, steer_loss: 0.047 
Epoch: 21, Train Loss: 2.795, aug_loss_primal: 0.122, aug_loss_fixed_point: 2.135, res_loss: 6.154, steer_loss: 0.044 
Epoch: 25, Train Loss: 2.694, aug_loss_primal: 0.101, aug_loss_fixed_point: 2.044, res_loss: 6.068, steer_loss: 0.043 
Epoch: 29, Train Loss: 2.624, aug_loss_primal: 0.082, aug_loss_fixed_point: 1.976, res_loss: 6.032, steer_loss: 0.044 
Epoch: 33, Train Loss: 2.528, aug_loss_primal: 0

KeyboardInterrupt: 

In [8]:
torch.save(model.state_dict(), './weights/learned_qcqp_model_1.pth')